## Faiss

### 설치

In [1]:
# !pip install -U langchain-community faiss-cpu langchain-openai tiktoken

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
    --------------------------------------- 0.0/2.3 MB 991.0 kB/s eta 0:00:03
   ------- -------------------------------- 0.5/2.3 MB 5.6 MB/s eta 0:00:01
   --------------- ------------------------ 0.9/2.3 MB 7.0 MB/s eta 0:00:01
   ----------------------- ---------------- 1.4/2.3 MB 7.8 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.3 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------  2.2/2.3 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/47.1 kB ? eta -:--:--
   ---------------------------------------- 47.1/47.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/377.3 kB ? eta -:--:--
   --------------------------------------  368.6/377.3 kB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 377.3/377.3 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: l

In [8]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

### DB에 저장

여기에서는 문서를 벡터 저장소에 수집합니다.

- TextLoader를 사용하여 텍스트 데이터를 로드합니다.
- CharacterTextSplitter를 사용하여 로드된 문서를 300 자 단위로 분할하고, 분할된 문서 간에 중복되는 내용이 없도록 설정합니다.
- OpenAIEmbeddings를 사용하여 문서 임베딩을 생성합니다.
- FAISS 벡터 저장소를 초기화하고, 분할된 문서와 임베딩을 사용하여 벡터 인덱스를 구축합니다.

참고

- AVX2: 고도의 병렬 처리가 가능한 연산을 사용하는 벡터화 가능 알고리즘의 경우 AVX2 를 사용하면 CPU 성능이 향상되어 지연 시간이 줄어들며 처리량이 향상됩니다.
- 필요한 경우 os.environ['FAISS_NO_AVX2'] = '1' 코드 라인의 주석을 해제하여 FAISS에서 AVX2 최적화를 사용하지 않도록 설정할 수 있습니다.

In [9]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# FAISS에서 AVX2 최적화를 사용하지 않으려면 다음 줄의 주석을 해제하세요.
# import os
#
# os.environ['FAISS_NO_AVX2'] = '1'

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, embeddings)

### 쿼리

In [10]:
# data/appendix-keywords.txt 파일 내용을 읽어서 file 변수에 저장합니다.
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

# file 변수에 저장된 내용을 출력합니다.
print(file[:500])

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어


In [11]:
# 쿼리(query) 변수에 저장된 질문과 유사한 문서를 데이터베이스에서 검색합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
docs = db.similarity_search(query)  # 질문과 유사한 문서를 데이터베이스에서 검색

In [12]:
# docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(docs[0].page_content)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


### Retriever로 활용

In [7]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = db.as_retriever()

In [8]:
# 검색 질의를 사용하여 관련 문서를 검색합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
docs = retriever.invoke(query)

In [9]:
# docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(docs[0].page_content)

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token


### 점수에 기반한 유사도 검색

In [10]:
# 쿼리와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.
docs_and_scores = db.similarity_search_with_score(query)
content, score = docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다
print("[Content]")
print(content.page_content)  # 선택된 문서의 page_content 속성을 출력합니다
print("\n[Score]")
print(score)  # 선택된 문서의 점수를 출력합니다

[Content]
정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

[Score]
0.29650295


In [11]:
# 질의를 임베딩 벡터로 변환합니다.
query = "임베딩(Embedding)이란 무엇인가요?"
embedding_vector = embeddings.embed_query(query)
# 임베딩 벡터를 사용하여 유사도 검색을 수행하고, 문서와 점수를 반환합니다.
docs_and_scores = db.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다

Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken')

### 저장 및 로드

In [12]:
# 로컬에 "MY_FIRST_DB_INDEX"라는 이름으로 데이터베이스를 저장합니다.
DB_INDEX = "MY_FIRST_DB_INDEX"
db.save_local(DB_INDEX)

In [13]:
# 로컬에 저장된 데이터베이스를 불러와 new_db 변수에 할당합니다.
new_db = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)

query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = new_db.similarity_search(query)

# 문서 리스트의 첫 번째 문서를 가져옵니다.
docs[0]

Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken')

### 바이트 형태로 직렬화/역직렬화

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("./data/appendix-keywords.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)

# HuggingFaceEmbeddings 사용하여 임베딩을 생성합니다.
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, hf_embeddings)

c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\anHye\anaconda3\envs\rag\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:

In [2]:
# FAISS 인덱스를 직렬화합니다.
serialized_db_index = db.serialize_to_bytes()

In [3]:
import sys

# 사이즈 측정을 위한 함수를 정의합니다.


def get_size(path):
    size = sys.getsizeof(path)
    if size < 1024:
        return f"{size} bytes"
    elif size < pow(1024, 2):
        return f"{round(size/1024, 2)} KB"
    elif size < pow(1024, 3):
        return f"{round(size/(pow(1024,2)), 2)} MB"
    elif size < pow(1024, 4):
        return f"{round(size/(pow(1024,3)), 2)} GB"


# 직렬화된 FAISS 인덱스의 사이즈를 출력합니다.
get_size(serialized_db_index)

'60.42 KB'

In [5]:
# 직렬화된 인덱스를 로드합니다.
deserialized_db = FAISS.deserialize_from_bytes(
    embeddings=hf_embeddings,  # 직렬화 할 때의 임베딩과 동일하게 지정
    serialized=serialized_db_index,  # 직렬화된 인덱스
    allow_dangerous_deserialization=True  # 위험한 디직렬화를 허용
)

In [6]:
query = "임베딩(Embedding)이란 무엇인가요?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = deserialized_db.similarity_search(query)

# 문서 리스트의 첫 번째 문서를 가져옵니다.
docs[0]

Document(metadata={'source': './data/appendix-keywords.txt'}, page_content='정의: JSON(JavaScript Object Notation)은 경량의 데이터 교환 형식으로, 사람과 기계 모두에게 읽기 쉬운 텍스트를 사용하여 데이터 객체를 표현합니다.\n예시: {"이름": "홍길동", "나이": 30, "직업": "개발자"}는 JSON 형식의 데이터입니다.\n연관키워드: 데이터 교환, 웹 개발, API\n\nTransformer')

### 병합(merge)

In [13]:
# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# db1 생성
db1 = FAISS.from_texts(["LangChain!!"], embeddings)
# db2 생성
db2 = FAISS.from_texts(["좋아요^^"], embeddings)

In [14]:
# db1 벡터 데이터베이스의 내부 문서 저장소 딕셔너리에 접근합니다.
db1.docstore._dict

{'3b8a020b-ba51-4235-94cf-4c34b3c73a1d': Document(page_content='LangChain!!')}

In [15]:
# db2 문서 저장소의 내부 딕셔너리에 접근합니다.
db2.docstore._dict

{'c613ab97-a9ac-45de-ab64-26416c60e52f': Document(page_content='좋아요^^')}

In [16]:
db1.merge_from(db2)  # db1에 db2를 병합합니다.

In [17]:
# db1 문서 저장소의 내부 딕셔너리에 접근합니다.
db1.docstore._dict

{'3b8a020b-ba51-4235-94cf-4c34b3c73a1d': Document(page_content='LangChain!!'),
 'c613ab97-a9ac-45de-ab64-26416c60e52f': Document(page_content='좋아요^^')}

### 필터링

In [18]:
from langchain_core.documents import Document

list_of_documents = [
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="foo", metadata=dict(page=1)),
    # 페이지 내용이 "bar"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="bar", metadata=dict(page=1)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="foo", metadata=dict(page=2)),
    # 페이지 내용이 "barbar"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="barbar", metadata=dict(page=2)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="foo", metadata=dict(page=3)),
    # 페이지 내용이 "bar burr"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="bar burr", metadata=dict(page=3)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="foo", metadata=dict(page=4)),
    # 페이지 내용이 "bar bruh"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
# 문서 리스트와 임베딩을 사용하여 FAISS 데이터베이스 생성
db = FAISS.from_documents(list_of_documents, embeddings)

# "foo"와 유사한 문서를 검색하고 점수와 함께 결과 반환
results_with_scores = db.similarity_search_with_score("foo")

for doc, score in results_with_scores:  # 검색 결과를 반복하면서
    # 각 문서의 내용, 메타데이터, 점수를 출력
    print(
        f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

Content: foo, Metadata: {'page': 1}, Score: 0.0
Content: foo, Metadata: {'page': 2}, Score: 0.0
Content: foo, Metadata: {'page': 3}, Score: 0.0
Content: foo, Metadata: {'page': 4}, Score: 0.0


In [19]:
# 방법1) 유사도 검색을 수행하고 필터를 적용하여 결과와 점수를 반환합니다.
results_with_scores = db.similarity_search_with_score("foo", filter=dict(page=1))
# 방법2) 혹은 callable 을 사용하여 필터링 하는 경우
# results_with_scores = db.similarity_search_with_score("foo", filter=lambda d: d["page"] == 1)

for doc, score in results_with_scores:  # 결과와 점수를 반복합니다.
    # 각 문서의 내용, 메타데이터, 점수를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}, [Score] {score}")

[Content] foo, [metadata] {'page': 1}, [Score] 0.0
[Content] bar, [metadata] {'page': 1}, [Score] 0.3147187829017639


### Max Marginal Relevance (MMR)

- db.max_marginal_relevance_search() 메서드를 사용하여 "foo"라는 검색어로 문서를 검색합니다.
- 이때 filter 매개변수를 사용하여 page 메타데이터 값이 1인 문서만 검색합니다.

In [20]:
# "foo"를 검색어로 사용하여 최대 한계 관련성 검색을 수행하고, 메타데이터의 'page' 필드가 1인 문서만 필터링합니다.
results = db.max_marginal_relevance_search("foo", filter=dict(page=1))

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")

[Content] foo, [metadata] {'page': 1}
[Content] bar, [metadata] {'page': 1}


In [21]:
results = db.similarity_search(
    "foo",  # 검색 쿼리
    # 메타데이터의 'page' 필드가 1인 문서만 필터링
    filter=dict(page=1),
    k=1,  # 가장 유사한 1개의 문서를 반환
    fetch_k=4,
)  # 4개의 문서까지 검색

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")

[Content] foo, [metadata] {'page': 1}


### 문서 삭제

In [22]:
db.index_to_docstore_id[0]

'f78c43b0-9fed-497c-80ff-fc1ea4bbe0c3'

In [23]:
# 인덱스 0에 해당하는 문서 저장소 ID를 사용하여 데이터베이스에서 문서를 삭제합니다.
db.delete([db.index_to_docstore_id[0]])

True